# Massachusetts income tax

Massachusetts has a mostly flat income tax of five percent, with personal exemptions based on filing status and number of dependents.
It also has other exemptions, deductions, and credits, including an Earned Income Tax Credit set at 30 percent of a filer's federal EITC payment.

## Examples

Consider a single person.

In [1]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

def make_tax(adults, children):    
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_household(name="household", members=members, state_code="MA")
    sim.vary("employment_income", max=100_000, step=1_000)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            state_income_tax=sim.calc("state_income_tax")[0].round(),
            mtr=sim.deriv("state_income_tax", "employment_income", wrt_target="head"),
            adults=adults,
            children=children,
        )
    )

# Make a table of state taxes for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_tax(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Monthly employment income",
    state_income_tax="Massachusetts income tax",
    mtr="State marginal tax rate",
    adults="Adults",
    children="Children",
)

fig = px.line(
    df,
    "employment_income",
    "state_income_tax",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Massachusetts income tax",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

ValueError: operands could not be broadcast together with shapes (202,) (101,) 

Core marginal tax rates span from zero to five percent, but when including the EITC, they range from -13.5% to 11.3%, depending on income and household structure.

In [ ]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Massachusetts marginal tax rate for a one-person household",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat=".1%")
fig.show()